In [105]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [106]:
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

In [107]:
num_classes = 10
n1 = 100
no_users = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [108]:
x_train = np.reshape(x_train, (x_train.shape[0], -1))/255
x_test = np.reshape(x_test, (x_test.shape[0], -1))/255

In [109]:
# Network parameters.
n_hidden_1 = 128 # 1st layer number of neurons.
n_hidden_2 = 64 # 2nd layer number of neurons.

In [110]:
W1 = tf.Variable(tf.random.truncated_normal([784,n_hidden_1], stddev=0.1))
W2 = tf.Variable(tf.random.truncated_normal([n_hidden_1,n_hidden_2], stddev=0.1))
W3 = tf.Variable(tf.random.truncated_normal([n_hidden_2,10], stddev=0.1))

In [111]:
class User:
    def __init__(self):       
        self.GW1 = tf.Variable(tf.random.truncated_normal([784,n_hidden_1], stddev=0.1))
        self.GW2 = tf.Variable(tf.random.truncated_normal([n_hidden_1,n_hidden_2], stddev=0.1))
        self.GW3 = tf.Variable(tf.random.truncated_normal([n_hidden_2,10], stddev=0.1))
        self.H = 1.0#np.random.normal()   
        self.abs_H_square = self.H**2
        

    def neural_net(self, x):
        y1 = tf.nn.relu(tf.matmul(x, W1))
        y2 = tf.nn.relu(tf.matmul(y1, W2))
        ylogits = tf.matmul(y2, W3)
        return tf.nn.softmax(ylogits)

In [112]:
# mini-batch loss function.
def mini_batches(X, Y, mb_size = 100):

    m = X.shape[0]

    perm = list(np.random.permutation(m))
    #perm = perm_init[0:100]
    X_temp = X[perm,:]
    Y_temp = Y[perm,:].reshape((m, Y.shape[1]))
    
    X_r = tf.convert_to_tensor(X_temp[0:mb_size,:], dtype=np.float32)
    Y_r = tf.convert_to_tensor(Y_temp[0:mb_size,:], dtype=np.float32)
    return X_r,Y_r

In [113]:
# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return -tf.reduce_sum(y_true * tf.math.log(y_pred))#tf.reduce_sum(tf.math.square(y_true-y_pred))#-tf.reduce_sum(y_true * tf.math.log(y_pred))
    #tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))
    #return tf.reduce_sum(-tf.reduce_mean(y_true * tf.math.log(y_pred)))

In [114]:
# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

In [115]:
users = [User() for i in range(no_users)]
optimizer = tf.optimizers.Adam(learning_rate=0.001)
central_modal = [tf.Variable(tf.random.truncated_normal([784*n_hidden_1,1], stddev=0.1)), tf.Variable(tf.random.truncated_normal([n_hidden_1*n_hidden_2,1], stddev=0.1)), tf.Variable(tf.random.truncated_normal([10*n_hidden_2,1], stddev=0.1))]

In [116]:
# Optimization process. 
def run_optimization(x, y, user, W1, W2, W3, rho):
    # Wrap computation inside a GradientTape for automatic differentiation.
    
    with tf.GradientTape() as g:
        g.watch([W1,W2,W3])
        pred = user.neural_net(x)
        loss = cross_entropy(pred, y) 
        
    
    # Variables to update, i.e. trainable variables.
    trainable_variables = [W1, W2 ,W3]
    
    # Compute gradients.
    gradients1,gradients2,gradients3  = g.gradient(loss, trainable_variables)
    #print(gradients2)
    return gradients1,gradients2,gradients3, loss
    #
    
    # Update W following gradients.
    # optimizer.apply_gradients(zip(gradients, trainable_variables))

In [117]:
def find_max(G):
    
    myMax = (tf.reduce_max(G)).numpy()
    #myMax=R1_max.eval()
    
    myMin = (tf.reduce_min(G)).numpy()
    #myMin=R1_min.eval()
    
    return myMax, myMin

In [118]:
x_train_k = []
y_train_k = []
data_per_worker = int(x_train.shape[0]/no_users)
for i in range(no_users):
    first = i*data_per_worker
    last = first + data_per_worker
    x_train_k.append(x_train[first:last])
    y_train_k.append(y_train[first:last])

In [119]:
x_test = tf.convert_to_tensor(x_test, dtype=np.float32)
y_test = tf.convert_to_tensor(y_test, dtype=np.float32)

In [120]:
R = tf.Variable(tf.zeros([1]))

In [121]:
import time

abs_weights_diff = []
abs_biases_diff = []
Train_Acc = []
Test_Acc = []
CrE_Train = []
CrE_Test = []
b=5
tau=1/(2**b-1)
lr = 10**(-2)

mb_size = 100
n_epochs = 500

acc_train = np.zeros([n_epochs,1])
acc_test = np.zeros([n_epochs,1])
total_loss = np.zeros([n_epochs,1])
H_matrix = np.zeros([int(n_epochs/10)])
eps = 10**(-6)

ii = 0

for k in range(n_epochs):
        print(k)
        
        batch_x = []
        batch_y = []
        for i in range(no_users):
            batch_xx , batch_yy = mini_batches(x_train_k[i],y_train_k[i],  mb_size)
            batch_x.append(batch_xx)
            batch_y.append(batch_yy)
        
        
        W1 = tf.reshape(central_modal[0],[784, n_hidden_1])
        W2 = tf.reshape(central_modal[1],[n_hidden_1, n_hidden_2])
        W3 = tf.reshape(central_modal[2],[n_hidden_2, 10])
        
        # Run the optimization to update W  
        for i in range(no_users):
            gradients1, gradients2, gradients3, loss = run_optimization(batch_x[i], batch_y[i], users[i], W1, W2, W3, lr)
            users[i].GW1 = gradients1
            users[i].GW2 = gradients2
            users[i].GW3 = gradients3
            
            G1 = tf.reshape(users[i].GW1,[784*n_hidden_1, 1])
            G2 = tf.reshape(users[i].GW2,[n_hidden_1*n_hidden_2, 1])
            G3 = tf.reshape(users[i].GW3,[10*n_hidden_2, 1])
                        
            [max_value, min_value]=find_max(G1)
            R[0].assign(max(abs(min_value), abs(max_value)))
            Q1=(G1+R[0])/(2*tau*R[0])+1/2
            Q1=tf.math.floor(Q1)#.eval())
            Q1= 2*tau*Q1*R[0]-R[0]
            users[i].GW1=tf.reshape(Q1,[784,n_hidden_1])
            
            [max_value, min_value]=find_max(G2)
            R[0].assign(max(abs(min_value), abs(max_value)))
            Q2=(G2+R[0])/(2*tau*R[0])+1/2
            Q2=tf.math.floor(Q2)#.eval())
            Q2= 2*tau*Q2*R[0]-R[0]
            users[i].GW2=tf.reshape(Q2,[n_hidden_1,n_hidden_2])
            
            [max_value, min_value]=find_max(G3)
            R[0].assign(max(abs(min_value), abs(max_value)))
            Q3=(G3+R[0])/(2*tau*R[0])+1/2
            Q3=tf.math.floor(Q3)#.eval())
            Q3= 2*tau*Q3*R[0]-R[0]
            users[i].GW3=tf.reshape(Q3,[n_hidden_2*10])
            
            total_loss[k] = total_loss[k] + loss
        
        total_loss[k] = total_loss[k]/no_users
        print(total_loss[k])
            
        sum_gradient_1 = 0
        sum_gradient_2 = 0
        sum_gradient_3 = 0
        for i in range(no_users):
            sum_gradient_1 = sum_gradient_1 + (users[i].GW1)
            sum_gradient_2 = sum_gradient_2 + (users[i].GW2)
            sum_gradient_3 = sum_gradient_3 + (users[i].GW3)
                

        sum_gradient_1 = tf.reshape(sum_gradient_1,[784*n_hidden_1, 1])
        sum_gradient_2 = tf.reshape(sum_gradient_2,[n_hidden_1*n_hidden_2, 1])
        sum_gradient_3 = tf.reshape(sum_gradient_3,[10*n_hidden_2, 1])
        
        central_modal[0] = central_modal[0] - lr*sum_gradient_1/no_users
        central_modal[1] = central_modal[1] - lr*sum_gradient_2/no_users
        central_modal[2] = central_modal[2] - lr*sum_gradient_3/no_users
        
        
        train_acc = []
        test_acc = []
        for j in range(no_users):  
            train_pred = users[j].neural_net(batch_x[j])
            train_acc.append(accuracy(train_pred , batch_y[j]))
            test_pred = users[j].neural_net(x_test)
            test_acc.append(accuracy(test_pred , y_test))
            
        avgAcc_Train = np.mean(train_acc)
        avgAcc_Test = np.mean(test_acc) 
        #print(avgAcc_Train)
        print(avgAcc_Test)
        acc_train[k] = avgAcc_Train
        acc_test[k] = avgAcc_Test

np.save('acc_test_qgd.npy', acc_test)
#np.save('loss_gd_40960_5.npy', total_loss)
#np.save('acc_train_gd_40960_5.npy', acc_train)
#np.save('acc_test_gd_40960_5.npy', acc_test)

0
[237.90766602]
0.1158
1
[215.36623535]
0.2827
2
[193.77445068]
0.514
3
[186.42475586]
0.3315
4
[283.83652344]
0.29500002
5
[214.57287598]
0.3076
6
[181.84833984]
0.4216
7
[154.76889648]
0.5091
8
[141.46520996]
0.5612999
9
[157.66259766]
0.47770005
10
[154.65856934]
0.5197
11
[156.75202637]
0.44349998
12
[134.00715332]
0.54969996
13
[131.43372803]
0.5512
14
[128.27862549]
0.5773
15
[141.3057251]
0.5143
16
[111.7651123]
0.6408
17
[101.34848022]
0.6889
18
[105.05865479]
0.6492
19
[109.34909668]
0.5867
20
[116.31396484]
0.5691
21
[86.35406494]
0.72389996
22
[64.74172363]
0.79129994
23
[63.10664673]
0.8002
24
[74.05291748]
0.7336999
25
[112.69796143]
0.6492
26
[100.12169189]
0.6571001
27
[84.66764526]
0.7311
28
[74.27041016]
0.7515
29
[61.72365112]
0.7789999
30
[48.45714111]
0.8557
31
[50.60075684]
0.8447
32
[54.41091309]
0.7993001
33
[55.71505737]
0.811
34
[63.95345459]
0.7992
35
[53.19732666]
0.8372
36
[51.81642456]
0.8456
37
[62.39022827]
0.82580006
38
[59.52015381]
0.8031
39
[48.35060

KeyboardInterrupt: 

In [94]:
np.save('acc_test_qgd_5bits.npy', acc_test)
#np.save('nb_ts_gd_lr001.npy', nb_ts)
